In [2]:
import pandas as pd
from glob import glob
import chardet
from tqdm import tqdm
import os
import json
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from models.model import LLM

데이터로드

In [3]:
RAW_DATA = sorted(glob('data/raw/*.csv'))

def extract_df(path):
    with open(path, 'rb') as f:
        result = chardet.detect(f.read())
    name, df = os.path.basename(path), pd.read_csv(path, encoding=result['encoding'])
    return name, df

dfs = [extract_df(i) for i in RAW_DATA]

데이터 전처리

In [24]:
queries = []
for name, df in tqdm(dfs):
    cols = list(df.columns)
    for _, rows in df.iterrows():
        text = ""
        for row, col in zip(rows, cols):
            text += col + " : " + str(row) + "\n"
        queries.append([name, text])
        text = ""

100%|██████████| 4/4 [00:00<00:00,  9.53it/s]


### LangGraph 세팅

In [25]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define Call model
def call_model(state: MessagesState):
    response = LLM.invoke(state["messages"])
    return {"messages": response}

# Set Node
workflow.add_node("model", call_model)

# Set Edge
workflow.add_edge(START, "model")

# Set Memory 
memory = MemorySaver()
# graph = workflow.compile(checkpointer=memory)
graph = workflow.compile()

# Set Config
config = {"configurable": {"thread_id": "abc124"}}

프롬프트

In [26]:
# Input
BASE_PROMPT = """
발전소 관련 데이터를 생성하고 있습니다.
용어에 대한 정의를 나타내는 문장을 생성하세요.

예를 들어, 아래와 같은 형식으로 주요 용어를 정의할 수 있습니다:
발전소(Power Plant)는 전기를 생산하기 위해 다양한 에너지원(화석 연료, 원자력, 재생 가능 에너지 등)을 이용하여 전력을 생성하고 이를 전력망에 공급하는 시설을 말한다. 발전소는 사용하는 에너지원과 발전 방식에 따라 여러 종류로 구분된다.

다음 지시사항을 따르세요.
1. 한국어와 영어를 제외한 언어는 사용하지 않습니다.
2. 정확히 아는 정보에 대해서는 자세히 설명합니다.
3. 생성해야할 용어에 대한 정보는 [용어 정보]를 참고하세요.

[용어 정보]
{query}

생성 문장 :
"""

데이터 생성 수행

In [33]:
SAVE_PATH = 'data/preprocessed/PowerPlant_Glossary.json'

try:
    dataset = json.load(open(SAVE_PATH, 'r', encoding='utf-8'))
except:
    dataset = []

for idx, query in enumerate(tqdm(queries[1892:9000])):
    PROMPT = BASE_PROMPT.format(query=query[1])
    INPUT_MESSAGES = [SystemMessage(content="당신은 데이터 생성 어시스턴트입니다."), 
                      HumanMessage(PROMPT)]
    try:
        output = graph.invoke({"messages":INPUT_MESSAGES}, config)["messages"][-1].content
        dataset.append({"name": query[0],
                        "info": query[1],
                        "prompt": PROMPT,
                        "response": output})
        with open(SAVE_PATH, 'w', encoding='utf-8') as file:
            json.dump(dataset, file, ensure_ascii=False, indent=4)
    except Exception as e:
        with open(f'error_log.txt', 'a', encoding='utf-8') as file:
            file.write(f"{idx} : {query[1]} : {e}\n")
        continue

100%|██████████| 7108/7108 [7:10:27<00:00,  3.63s/it]   
